In [ ]:
pip install jmespath parsel unidecode

In [ ]:
import json
import requests
import jmespath
from parsel import Selector

In [39]:

def SaveJSON(fileName,JsonComic):
  with open(f"{fileName}.json", "w") as outfile:
      json.dump(JsonComic, outfile,indent= 2)


def GetComicJSON(urlComic):
  response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/{urlComic}')
  assert (response.status_code == 200)
  selector = Selector(text=response.text)
  json_data = selector.css("script[type='application/json']::text").get()
  data = json.loads(json_data)
  # JMESPath expressions
  expression = jmespath.compile(
      """
      {
        "Title": props.pageProps.comic[0].nameComic,
        "URL": props.pageProps.comic[0].urlComic,
        "CoverImage":   props.pageProps.comic[0].imgComic,
        "CreateAt":  props.pageProps.comic[0].create_date
        "statusComic": props.pageProps.comic[0].statusComic
        "Category":   props.pageProps.comic[0].category[*].name_cate,
        "LastChapter": props.pageProps.comic[0].lastChapter
        "TotalChapter": props.pageProps.comic[0].totalChapter
        "Chapters": props.pageProps.listChapter[0].{urlChapter:urlChapter, dateUpdateChapter: dateUpdateChapter}
      }
      """
  )
  flat_data = expression.search(data)
  return flat_data





import time


In [41]:
ListComic = {"comic":[]}
def fetchData(start, end):
    global ListComic
    starting = True
    idx = 1
    expression = jmespath.compile(
        """
        {
        "comic": props.pageProps.dataComicNew[*].
        {
            urlComic:urlComic,
            totalChapter:totalChapter,
            createDateComic:createDateComic,
            lastUpdateChapter:lastUpdateChapter
            }
        }
        """
    )
    idx = start
    while idx<=end:
        print(f"fetching {idx}\n")
        # start = time.time()
        response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/tim-truyen?status=2&sort=2&page={idx}')
        idx+=1
        assert (response.status_code == 200)
        selector = Selector(text=response.text)
        json_data = selector.css("script[type='application/json']::text").get()
        data = json.loads(json_data)
        flat_data = expression.search(data)
        if(flat_data["comic"] == []): break
        ListComic["comic"].extend(flat_data["comic"])
        # end = time.time()
        # eplap = end - start
        # if(eplap <2):
        #     time.sleep(2 - eplap)

        # for oneComic in flat_data["comic"]:
            # urlComic = oneComic["urlComic"]
        #     JsonComic = GetComicJSON(urlComic)
            # SaveJSON(f"ComicData/{urlComic}",JsonComic)
    
    return ListComic

In [42]:
import threading
t1 = threading.Thread(target=fetchData, args=(1, 5))
t2 = threading.Thread(target=fetchData, args=(6, 10))
t3 = threading.Thread(target=fetchData, args=(11, 15))
t1.start()
t2.start()
t3.start()
t1.join()
t2.join()
t3.join()
SaveJSON("mapComic",ListComic)

fetching 1
fetching 6


fetching 11

fetching 12

fetching 2

fetching 7

fetching 13

fetching 3

fetching 8

fetching 14

fetching 4

fetching 9

fetching 15

fetching 5

fetching 10



In [ ]:
for oneComic in flat_data["comic"]:
  print(oneComic["urlComic"])

In [ ]:
GetChapterOfComicJSON("cuop-dau", chapter=1)


In [ ]:
def GetChapterOfComicJSON(urlComic, chapter=1):
  response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/{urlComic}/chapter-{chapter}')
  assert (response.status_code == 200)
  selector = Selector(text=response.text)
  json_data = selector.css("script[type='application/json']::text").get()
  data = json.loads(json_data)
  expression = jmespath.compile(
      """
      {
        "Page": props.pageProps.dataContentChapter[*].{indexImg:indexImg,imgChapter:imgChapter}
      }
      """
  )
  flat_data = expression.search(data)
  return flat_data

In [ ]:
JsonComic = GetComicJSON("toi-se-bi-tru-khu-cung-hoang-de")

In [ ]:
JsonComic

In [ ]:
# encodedUnicode = json.dumps(JsonComic, ensure_ascii=False)


In [ ]:
import unidecode
import re
def CreateSlug(accented_string:str):

  # accented_string is of type 'unicode'
  unaccented_string = unidecode.unidecode(accented_string).lower()
  unaccented_string = re.sub(r"[^\w\s]", '', unaccented_string)
  unaccented_string=re.sub(r"\s+","-",unaccented_string)
  return unaccented_string

CreateSlug("Tu Ti\u00ean \u1ede Th\u1ebf Gi\u1edbi Si\u00eau N\u0103ng L\u1ef1c")

In [ ]:
from bs4 import BeautifulSoup

# Assuming we have an HTML document in the 'html_content' variable
soup = BeautifulSoup(html_content, 'html.parser')

# Find the desired element and extract its text using get_text()
elements = soup.find_all('span',itemprop="name")

#get_text with strip set to true
for idx,element in  enumerate(elements):
  print(f"({idx},{element.text})")